In [1]:
import numpy as np
import networkx as nx
from scipy.linalg import sqrtm
import time

import perceval as pcvl
from perceval.algorithm import Sampler
from bs_bicluster_class import BS_bicluster

In [ ]:
#Load dataset
shots = 10**6
#fix columns to be selected for experiment 1
col_idx = [i for i in range(3,9)] #for experiment 1, we know which columns will our bicluster lie in
desired_output_state = [0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0] + [0 for i in range(12)] #for experiment 1, we know which rows will our bicluster lie in

batch_no = 1
for i in range(0,6):
    #load the file
    filename = 'bs_exp1_part'+str(i)+'.npy'
    filepath = './problems/' + filename
    dataset = np.load(filepath)

    #do the sampling
    bs_cluster = BS_bicluster(dataset)#,'quandela_token.txt')
    bs_cluster.prepare_backend()#('sim:clifford')
    print("Unitary error:",bs_cluster.unitary_error)
    raw_samples = bs_cluster.boson_sampling(col_idx,shots)

    #quick analysis

    #save the results
    outfilename= 'res_bs_exp1_part'+str(i)+'_batch'+str(batch_no)+'.json'
    outfilepath = './results/' + outfilename
    bs_cluster.save_samples(outfilepath)

    #quick analysis
    # (Probability, numerator, denominator)
    print("Desired probability (basic):",bs_cluster.get_probability(desired_output_state,row_postselect=False))
    print("Desired probability (postselected):",bs_cluster.get_probability(desired_output_state,row_postselect=True))


    #Sleep for sometime between 2 and 4 seconds before next api call
    random_seconds = 2*np.random.rand() + 2
    print(f"Sleeping for {random_seconds} seconds")
    time.sleep(random_seconds)
    print("----")


Unitary error: 2.942330327745671e-08


In [ ]:
np.linalg.norm(bs_cluster.U @ np.conjugate(bs_cluster.U).T - np.identity(24))

In [7]:
bs_cluster.save_samples('exp1_temp.json')

In [8]:
bs_cluster.get_probability(desired_output_state,row_postselect=True)

Desired modes: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]
Undesired modes: [12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23]


(0.84375, 27, 32)

In [9]:
bs_cluster.get_probability(desired_output_state)

(0.00027000270002700027, 27, 99999)

In [4]:
from  perceval.utils.postselect import PostSelect

In [75]:
desired_modes = [0,1,2,3]
undesired_modes = [4,5,6,7]

def condition_builder(desired_modes, undesired_modes, max_photon_per_mode = 1):
    #desired_modes and undesired_modes are mode numbers mentioned in the list
    tempstr = ''
    first = True

    #desired
    for item in desired_modes:
        if first == True:
            tempstr += '[' + str(item) + '] < ' + str(max_photon_per_mode + 1)
            first = False
        else:
            tempstr += ' & '
            tempstr += '[' + str(item) + '] < ' + str(max_photon_per_mode + 1)
    
    #undesired
    for item in undesired_modes:
        if first == True:
            tempstr += '[' + str(item) + '] == 0 '
            first = False
        else:
            tempstr += ' & '
            tempstr += '[' + str(item) + '] == 0'

    return tempstr

            
   
ps = PostSelect(condition_builder(desired_modes,undesired_modes)) #I don't want any photons in modes 4 to 7 and I want 1 or 0 photons in modes 0 to 3

In [76]:
condition_builder([],undesired_modes)  

'[4] == 0  & [5] == 0 & [6] == 0 & [7] == 0'

In [86]:
ps(pcvl.BasicState('|0,0,1,1,0,0,1,0>'))

False

In [21]:
str(undesired_modes)+ ' == 0 & '+str(desired_modes) +' < 2'

'[4, 5, 6, 7] == 0 & [0, 1, 2, 3] < 2'

In [10]:
import bosonsampling as bs

In [11]:
bs.output_probability([0,0,0,1,1,1,1,1,1,0,0,0], [0,0,0,1,1,1,1,1,1,0,0,0], dataset)

28569.700782547585

In [14]:
_, D, _ = np.linalg.svd(dataset)
c = np.max(D)
new_dataset = dataset/c

bs.output_probability([0,0,0,1,1,1,1,1,1,0,0,0], [0,0,0,1,1,1,1,1,1,0,0,0], new_dataset)

0.00021764455724579288

In [20]:
photons_in= [0,0,0,1,1,1,1,1,1,0,0,0]
max_prob = 0
op_config = None
for photon_out_configuration in bs.gen_output_configurations(sum(photons_in), len(photons_in)):

    output_probability = bs.output_probability(photons_in, photon_out_configuration, new_dataset)
    if max_prob < output_probability:
        max_prob = output_probability
        op_config = photon_out_configuration.copy()
    print("Probability of photon output {0} : {1}".format(photon_out_configuration, output_probability))



Probability of photon output [0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1] : 9.153758936088673e-12
Probability of photon output [0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1] : 1.06341394540582e-11
Probability of photon output [0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1] : 1.1154966064091854e-11
Probability of photon output [0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1] : 1.0640402114184014e-11
Probability of photon output [0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1] : 2.921192137126275e-09
Probability of photon output [0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1] : 5.484222946309311e-09
Probability of photon output [0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0] : 1.4626261881270062e-09
Probability of photon output [0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1] : 9.173839336436508e-12
Probability of photon output [0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1] : 9.584390411587544e-12
Probability of photon output [0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1] : 9.153758936089328e-12
Probability of photon output [0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1] : 2.6333114394344443e-09
Probabi

In [18]:
max_prob

0.00021764455724579288

In [21]:
op_config

[0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0]

this is 3 seconds


2.976632661700786